# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../output_data/cities.csv")
weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.52,-28.70,62.60,72,40,8.05,PT,1592169751
1,1,faya,18.39,42.45,75.20,46,97,1.12,SA,1592169751
2,2,qaanaaq,77.48,-69.36,41.83,69,0,5.88,GL,1592169751
3,3,hermanus,-34.42,19.23,52.00,96,89,1.07,ZA,1592169751
4,4,mahina,-17.51,-149.49,84.20,66,20,4.70,PF,1592169751
...,...,...,...,...,...,...,...,...,...,...
566,566,rio gallegos,-51.62,-69.22,39.20,86,90,5.82,AR,1592169776
567,567,roma,43.21,-75.46,64.99,45,90,3.36,US,1592169776
568,568,rundu,-17.93,19.77,52.99,44,0,4.99,NaN,1592169776
569,569,bairiki,1.33,172.98,84.20,70,20,5.82,KI,1592169776


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Filla NaN values and convert to float
humidity = weather_df["Humidity"].astype(float)

In [4]:
gmaps.configure(api_key=g_key)

fig =gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, point_radius=4)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
idealWeather =  weather_df[(weather_df['Max Temp'] > 70) & 
                           (weather_df['Max Temp'] < 80) &
                           (weather_df['Wind Speed'] < 10) & 
                           (weather_df['Cloudiness'] == 0)]
idealWeather.count()
idealWeather
# dropna

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,49,changli,39.70,119.15,73.00,23,0,6.60,CN,1592169753
68,68,torbat-e jam,35.24,60.62,78.69,16,0,1.10,IR,1592169753
136,136,caravelas,-17.71,-39.25,72.72,79,0,4.07,BR,1592169661
184,184,gat,31.61,34.76,75.00,78,0,5.14,IL,1592169758
199,199,srednekolymsk,67.45,153.68,75.02,34,0,9.75,RU,1592169759
356,356,korla,41.76,86.15,72.00,37,0,3.00,CN,1592169766
375,375,galatas,37.50,23.45,79.00,59,0,5.48,GR,1592169766
399,399,barda,40.37,47.13,79.72,37,0,2.15,AZ,1592169767


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = idealWeather
# # find the closest restaurant of each type to coordinates
hotel_df     

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,49,changli,39.70,119.15,73.00,23,0,6.60,CN,1592169753
68,68,torbat-e jam,35.24,60.62,78.69,16,0,1.10,IR,1592169753
136,136,caravelas,-17.71,-39.25,72.72,79,0,4.07,BR,1592169661
184,184,gat,31.61,34.76,75.00,78,0,5.14,IL,1592169758
199,199,srednekolymsk,67.45,153.68,75.02,34,0,9.75,RU,1592169759
356,356,korla,41.76,86.15,72.00,37,0,3.00,CN,1592169766
375,375,galatas,37.50,23.45,79.00,59,0,5.48,GR,1592169766
399,399,barda,40.37,47.13,79.72,37,0,2.15,AZ,1592169767


In [7]:
hotel_df["Hotel Name"] = ""
hotel_df

totalCities = len(hotel_df)
totalCities
# lat = hotel_df["Lat"]
# lng = hotel_df["Lng"]

# #creating base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key}

count=1
#looping
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"
    # assemble url and make API request
    print(f"Index {count} of {totalCities}: {city_name}")
    response = requests.get(base_url, params=params).json()
    # extract results
    results = response['results']
    try:
        
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        count+=1
    # exception handling
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    print("------------")
# Print end of search once searching is completed
print("-------End of Search-------")



C:\Users\kathl\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Index 1 of 8: changli


C:\Users\kathl\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel in changli is Hesheng Hotel.
------------
Index 2 of 8: torbat-e jam
Closest hotel in torbat-e jam is Pardisan Hotel 🏩.
------------
Index 3 of 8: caravelas
Closest hotel in caravelas is Pousada Liberdade.
------------
Index 4 of 8: gat
Closest hotel in gat is Desert Gat.
------------
Index 5 of 8: srednekolymsk
Missing field/result... skipping.
------------
Index 5 of 8: korla
Closest hotel in korla is Kangcheng Jian Guo International Hotel.
------------
Index 6 of 8: galatas
Closest hotel in galatas is Golden View Beach Hotel.
------------
Index 7 of 8: barda
Closest hotel in barda is Bərdə Hotel.
------------
-------End of Search-------


In [9]:
hotel_df = hotel_df[hotel_df['Hotel Name'] != ""]
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
49,49,changli,39.70,119.15,73.00,23,0,6.60,CN,1592169753,Hesheng Hotel
68,68,torbat-e jam,35.24,60.62,78.69,16,0,1.10,IR,1592169753,Pardisan Hotel 🏩
136,136,caravelas,-17.71,-39.25,72.72,79,0,4.07,BR,1592169661,Pousada Liberdade
184,184,gat,31.61,34.76,75.00,78,0,5.14,IL,1592169758,Desert Gat
356,356,korla,41.76,86.15,72.00,37,0,3.00,CN,1592169766,Kangcheng Jian Guo International Hotel
375,375,galatas,37.50,23.45,79.00,59,0,5.48,GR,1592169766,Golden View Beach Hotel
399,399,barda,40.37,47.13,79.72,37,0,2.15,AZ,1592169767,Bərdə Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))